In [1]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import glob
import shutil
import pprint 
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import time
import pprint as pp
import collections
import scipy


pp = pprint.PrettyPrinter()

Using TensorFlow backend.


In [2]:
# Create Image Directory from Train.bson
# Set Download to False if directory has already been created

Download = False
start = time.time()

if Download:
    
    data = bson.decode_file_iter(open('/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount_bson_files/train.bson', 'rb'))

    prod_to_category = dict()

    picture_pipeline = []

    for c,d in enumerate(data):
        picture=[]
        category_id = d['category_id']
        number_pics = len(d['imgs'])
        prod_to_category[category_id] = number_pics


        directory = '/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount/Images'+str(category_id)+'/'

        if not os.path.exists(directory):
            os.makedirs(directory)
            for e, pic in enumerate(d['imgs']):
                picture = imread(io.BytesIO(pic['picture']))
                scipy.misc.imsave(str(directory)+str(category_id)+str(e)+'.png', picture)       
        elif os.path.exists(directory):
            for e, pic in enumerate(d['imgs']):
                picture = imread(io.BytesIO(pic['picture']))
                scipy.misc.imsave(str(directory)+str(category_id)+str(e)+'.png', picture)       
        else:
            pass


    print len(prod_to_category)
    print len(set(prod_to_category.keys()))

    Img_Down_time = time.time()

    print "Time taken for downloading the images in minutes is: " (Img_Down_time - start)/60.0

In [3]:
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return files,targets


In [4]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt

Augmentation = False

if Augmentation:
    Pre_Aug_Files, Pre_Aug_Labels = load_dataset("/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount/")
    Category_folder=[]
    for i in Pre_Aug_Files:
        Category_folder.append(i.split("/")[-2])
    print len(Category_folder)
    
    datagen = ImageDataGenerator(rotation_range=90,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    
    for i in range(len(Pre_Aug_Files)):
        img = load_img(Pre_Aug_Files[i])  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        
        if not os.path.exists("/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount/"+str(Category_folder[i])):
            os.makedirs("/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount/"+str(Category_folder[i]))
            
        count=0
        pointer = 4
        for batch in datagen.flow(x, batch_size=1,save_to_dir="/media/raj/96661FA2661F81E3/DOWNLOADS/Cdiscount/"+str(Category_folder[i]),save_prefix=str(Category_folder[i])+"_aug", save_format='png'):
            pointer += 1
            count += 1
            if count > 20:
                break  # otherwise the generator would loop indefinitely

In [5]:
def Move_Images_png(From_dir,Folder_name,Num_images):
    From_path = From_dir + "*.png" if From_dir[-1]=="/" else From_dir + "/*.png"
    To_path = From_dir.replace("Train",Folder_name)
    To_path = To_path+"/" if To_path[-1]!="/" else To_path
    if not os.path.exists(To_path):
        os.makedirs(To_path)
    x = glob.glob(From_path)
    for i in x[0:Num_images-1]:
        shutil.move(i,To_path+str(i.split("/")[-1]))

In [6]:
Move = False

if Move:
    for i in glob.glob("./../Cdiscount/Train/*/"):
        Move_Images_png(i,'Validation',2)

In [7]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(180, 180))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [8]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset('/home/raj/Projects/Capstone/Image-Classification/Cdiscount1/Train')
valid_files, valid_targets = load_dataset('/home/raj/Projects/Capstone/Image-Classification/Cdiscount1/Validation')
#test_files, test_targets = load_dataset('./Cdiscount/Test')


In [9]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 20/20 [00:00<00:00, 631.15it/s]


In [10]:
print train_tensors.shape
#print train_targets[0]

(54, 180, 180, 3)


In [11]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential



In [12]:
from keras.callbacks import ModelCheckpoint  

train_from_scratch = False

if train_from_scratch== True:
    
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=2,padding='same',activation='relu',input_shape=(180,180,3)))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=32, kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(filters=64, kernel_size=2,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(20, activation='softmax'))
    ### TODO: Define your architecture.
    
    model.summary()
    
    model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
    
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(train_tensors)

    epochs = 100


    if not os.path.exists("./saved_models_20_rmsprop"):
        os.makedirs("./saved_models_20_rmsprop")
    checkpointer = ModelCheckpoint(filepath='saved_models_20_rmsprop/weights.best.from_scratch.hdf5', 
                                   verbose=1, save_best_only=True)


    # fits the model on batches with real-time data augmentation:
    model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=32),
                        validation_data=(valid_tensors, valid_targets),
                        steps_per_epoch=len(train_tensors) / 1, epochs=epochs,
                        callbacks=[checkpointer],verbose=1)

    """
    # here's a more "manual" example
    for e in range(epochs):
        print('Epoch', e)
        batches = 0
        for x_batch, y_batch in datagen.flow(train_tensors, train_targets, batch_size=32):
            model.fit(x_batch, y_batch)
            batches += 1
            if batches >= len(train_tensors) / 32:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break
    """

In [13]:

"""
Object_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]


# report test accuracy
test_accuracy = 100*np.sum(np.array(Object_predictions)==np.argmax(test_targets, axis=1))/len(Object_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

"""

"\nObject_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]\n\n\n# report test accuracy\ntest_accuracy = 100*np.sum(np.array(Object_predictions)==np.argmax(test_targets, axis=1))/len(Object_predictions)\nprint('Test accuracy: %.4f%%' % test_accuracy)\n\n"

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=0.2,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(train_tensors)

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 20 classes
predictions = Dense(20, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='Nadam', loss='categorical_crossentropy',metrics=['accuracy'])

# train the model on the new data for a few epochs
epochs = 20


if not os.path.exists("./saved_models_20_inception"):
    os.makedirs("./saved_models_20_inception")
checkpointer = ModelCheckpoint(filepath='saved_models_20_inception/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)


model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=32),
                    validation_data=(valid_tensors, valid_targets),
                    steps_per_epoch=len(train_tensors) / 1, epochs=epochs,
                    callbacks=[checkpointer],verbose=1)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)



Epoch 1/20
54/54 [==============================] - 102s - loss: 0.7815 - acc: 0.8382 - val_loss: 3.9366 - val_acc: 0.4000
Epoch 2/20
54/54 [==============================] - 99s - loss: 0.0149 - acc: 0.9986 - val_loss: 3.8699 - val_acc: 0.2500
Epoch 3/20
54/54 [==============================] - 98s - loss: 2.7598 - acc: 0.7784 - val_loss: 7.0679 - val_acc: 0.4500
Epoch 4/20
54/54 [==============================] - 100s - loss: 0.4136 - acc: 0.9607 - val_loss: 4.9140 - val_acc: 0.3500
Epoch 5/20
54/54 [==============================] - 101s - loss: 0.0094 - acc: 0.9986 - val_loss: 5.9242 - val_acc: 0.3000
Epoch 6/20
54/54 [==============================] - 99s - loss: 0.0083 - acc: 0.9972 - val_loss: 4.3656 - val_acc: 0.3500
Epoch 7/20
54/54 [==============================] - 98s - loss: 0.0044 - acc: 0.9992 - val_loss: 4.3383 - val_acc: 0.3500
Epoch 8/20
54/54 [==============================] - 100s - loss: 9.1422e-04 - acc: 1.0000 - val_loss: 4.3606 - val_acc: 0.3500
Epoch 9/20
54/54

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

epochs=100
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=32),
                    validation_data=(valid_tensors, valid_targets),
                    steps_per_epoch=len(train_tensors) / 1, epochs=epochs,
                    callbacks=[checkpointer],verbose=1)

Epoch 1/100
54/54 [==============================] - 118s - loss: 0.0031 - acc: 0.9983 - val_loss: 4.9027 - val_acc: 0.3500
Epoch 2/100
54/54 [==============================] - 116s - loss: 5.6564e-04 - acc: 1.0000 - val_loss: 4.9410 - val_acc: 0.3500
Epoch 3/100
54/54 [==============================] - 117s - loss: 0.0030 - acc: 0.9992 - val_loss: 4.9430 - val_acc: 0.3500
Epoch 4/100
54/54 [==============================] - 116s - loss: 3.6360e-04 - acc: 1.0000 - val_loss: 4.9207 - val_acc: 0.3500
Epoch 5/100
54/54 [==============================] - 117s - loss: 6.5883e-04 - acc: 1.0000 - val_loss: 4.9527 - val_acc: 0.3500
Epoch 6/100
54/54 [==============================] - 116s - loss: 1.6395e-04 - acc: 1.0000 - val_loss: 4.8925 - val_acc: 0.3500
Epoch 7/100
54/54 [==============================] - 116s - loss: 3.7497e-04 - acc: 1.0000 - val_loss: 4.8929 - val_acc: 0.3500
Epoch 8/100
54/54 [==============================] - 116s - loss: 0.0022 - acc: 0.9992 - val_loss: 4.9528 - val_

54/54 [==============================] - 116s - loss: 4.5445e-04 - acc: 1.0000 - val_loss: 4.9383 - val_acc: 0.3500
Epoch 35/100
54/54 [==============================] - 115s - loss: 0.0012 - acc: 1.0000 - val_loss: 4.8967 - val_acc: 0.3500
Epoch 36/100
54/54 [==============================] - 115s - loss: 1.5119e-04 - acc: 1.0000 - val_loss: 4.9002 - val_acc: 0.3500
Epoch 37/100
54/54 [==============================] - 116s - loss: 0.0015 - acc: 0.9992 - val_loss: 4.9101 - val_acc: 0.3500
Epoch 38/100
54/54 [==============================] - 115s - loss: 2.4151e-04 - acc: 1.0000 - val_loss: 4.9135 - val_acc: 0.3500
Epoch 39/100
54/54 [==============================] - 115s - loss: 5.8182e-04 - acc: 1.0000 - val_loss: 4.9209 - val_acc: 0.3500
Epoch 40/100
26/54 [=============>................] - ETA: 59s - loss: 1.6792e-04 - acc: 1.0000